# Utilities
> Build utilities for flask

In [1]:
# default_exp utils

In [2]:
# export
import os
from pathlib import Path
import json
import pandas as pd

In [3]:
# export
HOME = Path(os.environ["HOME"])
EMBER = HOME/".torchember"
DATA = EMBER/"data"
LOG = EMBER/"LOG"
def check_existance():
    if DATA.exists()==False:
        return False
    if LOG.exists()==False:
        return False
    else:
        return True
    

Check if the data file exists, if not, use torch ember in your pytorch modeling first according to the short tutorial

In [4]:
check_existance()

True

In [5]:
# export
def get_ember_list():
    if check_existance()== False:
        return None
    else:
        return  list(i for i in os.listdir(DATA) if i[:5]=="base_")

In [6]:
get_ember_list()[:5]

['base_AlexNet_20200215_132629.json',
 'base_AlexNet_20200211_230327.json',
 'base_AlexNet_20200211_192928.json',
 'base_AlexNet_20200209_183014.json',
 'base_AlexNet_20200211_193437.json']

In [7]:
# export
def unpack_meta(fname):
    f = open(DATA/fname,"r")
    dict_ = json.loads(f.read())
    f.close()
    dict_["name"] = fname[5:-5]
    return dict_

def get_ember_df(ember_list):
    df = pd.DataFrame(list(unpack_meta(i) for i in ember_list))
    df = df.sort_values(by = "start",ascending = False)
    return df.reset_index().drop("index",axis=1)

In [8]:
df = get_ember_df(get_ember_list())
df.head(5)

,start,user,name
0,2020-02-24 23:28:27,salvor,AlexNet_20200224_232827
1,2020-02-24 22:46:54,salvor,AlexNet_20200224_224654
2,2020-02-24 21:40:30,salvor,AlexNet_20200224_214030
3,2020-02-20 00:32:06,salvor,tinyVGG_20200220_003206
4,2020-02-20 00:30:33,salvor,tinyVGG_20200220_003033


### Ember Reader
A reading handler to process Torch Ember data

In [11]:
# export
from torchember.core import emberTracker

class emberReader(object):
    def __init__(self, name):
        self.name = name
        self.t = emberTracker(name)
        self.structure = self.t[f"structure_{self.name}"]
        self.base = self.t[f"base_{self.name}"]
        
    @property
    def latest(self):
        return self.t.latest_df.to_dict(orient = "record")

In [27]:
er = emberReader("AlexNet_20200224_233207")

In [28]:
er.structure

{'name': 'model(AlexNet)',
 'short': 'model(AlexNet)',
 'children': [{'name': 'model(AlexNet).features(Sequential)',
   'short': 'features(Sequential)',
   'children': [{'name': 'model(AlexNet).features(Sequential).0(Conv2d)',
     'short': '0(Conv2d)'},
    {'name': 'model(AlexNet).features(Sequential).1(ReLU)',
     'short': '1(ReLU)'},
    {'name': 'model(AlexNet).features(Sequential).2(MaxPool2d)',
     'short': '2(MaxPool2d)'},
    {'name': 'model(AlexNet).features(Sequential).3(Conv2d)',
     'short': '3(Conv2d)'},
    {'name': 'model(AlexNet).features(Sequential).4(ReLU)',
     'short': '4(ReLU)'},
    {'name': 'model(AlexNet).features(Sequential).5(MaxPool2d)',
     'short': '5(MaxPool2d)'},
    {'name': 'model(AlexNet).features(Sequential).6(Conv2d)',
     'short': '6(Conv2d)'},
    {'name': 'model(AlexNet).features(Sequential).7(ReLU)',
     'short': '7(ReLU)'},
    {'name': 'model(AlexNet).features(Sequential).8(Conv2d)',
     'short': '8(Conv2d)'},
    {'name': 'model(AlexN

In [29]:
er.base

{'start': '2020-02-24 23:32:07', 'user': 'salvor'}

In [30]:
er.t.latest_df

,shape,mean,std,max,min,cnt_zero,module,ts,ttype,tname
0,"[2, 3, 224, 224]",-1.000411e+00,0.577138,-0.000004,-1.999977,0,model(AlexNet),2020-02-24 23:32:14,input,x
1,"[2, 3, 224, 224]",-1.000411e+00,0.577138,-0.000004,-1.999977,0,model(AlexNet).features(Sequential),2020-02-24 23:32:14,input,input
2,"[2, 3, 224, 224]",-1.000411e+00,0.577138,-0.000004,-1.999977,0,model(AlexNet).features(Sequential).0(Conv2d),2020-02-24 23:32:14,input,input
3,"[64, 3, 11, 11]",1.686268e-05,0.030325,0.052485,-0.052482,0,model(AlexNet).features(Sequential).0(Conv2d),2020-02-24 23:32:14,weight,weight_0
4,"[64, 3, 11, 11]",-6.141816e-07,0.000025,0.000098,-0.000095,363,model(AlexNet).features(Sequential).0(Conv2d),2020-02-24 23:32:14,weight_grad,grad_0
...,...,...,...,...,...,...,...,...,...,...
75,[1000],-3.101853e-04,0.009026,0.015589,-0.015622,0,model(AlexNet).classifier(Sequential).6(Linear),2020-02-24 23:32:16,weight,weight_1
76,[1000],2.000000e-03,0.000000,0.002000,0.002000,0,model(AlexNet).classifier(Sequential).6(Linear),2020-02-24 23:32:16,weight_grad,grad_1
77,"[2, 1000]",-3.422117e-04,0.010874,0.031096,-0.032036,0,model(AlexNet).classifier(Sequential).6(Linear),2020-02-24 23:32:16,output,output_0
78,"[2, 1000]",-3.422117e-04,0.010874,0.031096,-0.032036,0,model(AlexNet).classifier(Sequential),2020-02-24 23:32:16,output,output_0


In [31]:
er.latest

[{'shape': [2, 3, 224, 224],
  'mean': -1.0004112720489502,
  'std': 0.5771379470825195,
  'max': -3.814697265625e-06,
  'min': -1.999977469444275,
  'cnt_zero': 0,
  'module': 'model(AlexNet)',
  'ts': '2020-02-24 23:32:14',
  'ttype': 'input',
  'tname': 'x'},
 {'shape': [2, 3, 224, 224],
  'mean': -1.0004112720489502,
  'std': 0.5771379470825195,
  'max': -3.814697265625e-06,
  'min': -1.999977469444275,
  'cnt_zero': 0,
  'module': 'model(AlexNet).features(Sequential)',
  'ts': '2020-02-24 23:32:14',
  'ttype': 'input',
  'tname': 'input'},
 {'shape': [2, 3, 224, 224],
  'mean': -1.0004112720489502,
  'std': 0.5771379470825195,
  'max': -3.814697265625e-06,
  'min': -1.999977469444275,
  'cnt_zero': 0,
  'module': 'model(AlexNet).features(Sequential).0(Conv2d)',
  'ts': '2020-02-24 23:32:14',
  'ttype': 'input',
  'tname': 'input'},
 {'shape': [64, 3, 11, 11],
  'mean': 1.6862675693118945e-05,
  'std': 0.030325137078762054,
  'max': 0.05248480290174484,
  'min': -0.0524822361767292